## Fourier Transforms

**Nick Kern**
<br>
**Astro 9: Python Programming in Astronomy**
<br>
**UC Berkeley**

Reading: Chp 7 of Newman

The [Fourier Transform](https://en.wikipedia.org/wiki/Fourier_transform) is a mathematical operation that is one of the most widely used techniques in physics and engineering. A Fourier Transform allows us to take a time-dependent signal and transform it into frequency space. You may have seen the Fourier Transform defined as an integral over a continuous field:

\begin{align}
F(\nu) &= \int_{-\infty}^{\infty}f(t)e^{-2\pi i\nu t}dt\\
\\
f(t) &= \int_{-\infty}^{\infty}F(\nu)e^{2\pi i\nu t}d\nu
\end{align}

In this lecture, we will look at how we can approximate the above integrals on computers. As you may have guessed, this means approximating continuous integrals as discrete sums.

### The Fourier Series

At the core of a Fourier Transform (FT) is the concept of the [Fourier Series](https://en.wikipedia.org/wiki/Fourier_series), which says that we can represent any periodic signal as the infinite linear sum of sinusoids with varying amplitudes and wavelengths. Let's assume we have some periodic signal on the $x$-axis, and we are interested in looking at its behavior from $-L/2 < x < L/2$.

<img src='imgs/sawtooth.png' width=500px/>
<center>A sawtooth wave, which we will look to model from $-L/2 < x < L/2$.

Because we have both the cosine and sine functions, we can write two distinct Fourier Series for different kinds of periodic signals using either cosines or sines. Because cosine is symmetric about the y-axis, we call this an *even* function, and because sine is symmetric about the origin, we call this an *odd* function. 

<img src='imgs/sinusoids.png' width=400px/>

We can therefore construct an even and odd Fourier series, which looks something like

\begin{align}
f(x) &= \sum_{k=0}^{n=\infty}\alpha_k\cos\left(\frac{2\pi kx}{L}\right)\\
\\
f(x) &= \sum_{k=1}^{n=\infty}\beta_k\sin\left(\frac{2\pi kx}{L}\right)
\end{align}

For a periodic signal that is odd we would use the sine series, and for a signal that is even we would use the cosine series. The only trick is to figure out the values for the scalars $\alpha_k$ and $\beta_k$.

### Breakout 1: Sine Series for Sawtooth Wave

Try to recreate the sawtooth function with a sine series. Assume a sawtooth function can be represented analytically as $f(x) = \frac{x}{L}$ for $-\tfrac{L}{2} < x < \tfrac{L}{2}$, where you should take $L = 1$. Further, use the fact that the coefficients $\beta_k$ for a sawtooth wave are given as

\begin{align}
\beta_k = -\frac{2A}{k\pi}(-1)^k,
\end{align}

where $A$ is the amplitude of the sawtooth function, which for us is $A=0.5$.

1.
Write a function that will take-in a value for $x$ and will output $f(x)$ given the number of terms to use in the series, $n$.

2.
Make a plot of the sawtooth and then overplot the sine series with $n=[1, 3, 5, 10, 100]$.

In [87]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

In [88]:
def sine_series(x, n=100):

    return 

In general, though, some signal we want to represent is neither odd nor even. For such an arbitrary function, we can use a combination of both sines and cosines:

\begin{align}
f(x) = \sum_{k=0}^{\infty}\alpha_k\cos\left(\frac{2\pi kx}{L}\right) + \sum_{k=1}^{\infty}\beta_k\sin\left(\frac{2\pi kx}{L}\right).
\end{align}

Having to write out the sine and cosine terms is a little cumbersome. We can simplify it by turning the cosine and sine terms into a single complex term using [**Euler's Formula**](https://en.wikipedia.org/wiki/Euler%27s_formula), which says $e^{ix} = \cos(x) + i\sin(x)$. Utilizing the identities of $\cos\theta = \tfrac{1}{2}\left(e^{-i\theta}+e^{i\theta}\right)$ and $\sin\theta = \tfrac{1}{2}i\left(e^{-i\theta}-e^{i\theta}\right)$, we find that the previous equation can also be represented as

\begin{align}
f(x) &= \sum_{-\infty}^{\infty}\gamma_k\exp\left(i\frac{2\pi kx}{L}\right)\\
\text{where}&\\
\gamma_k &=
\begin{cases}
\tfrac{1}{2}(\alpha_{-k}+i\beta_{-k}) & \text{if}\ k\lt0\\
\alpha_0 & \text{if}\ k = 0\\
\tfrac{1}{2}(\alpha_k - i\beta_k) & \text{if}\ k\gt0
\end{cases}
\end{align}

Note that we only have a single term in our sum, but the sum now goes from $-\infty \rightarrow \infty$.

The question now becomes how we solve for the $\gamma_k$ terms. We can do this by **integrating our sinusoids against the function**, like so:

\begin{align}
y_k = \frac{1}{L}\int f(x)\exp\left(-i\frac{2\pi kx}{L}\right)dx
\end{align}

It is not unusual that $f(x)$ is such a complicated function that we cannot perform the above integral by hand. The question then becomes, how can we do this numerically? To do this, we need to use **numerical integration**. This is also known as the **discrete Fourier transform**.

### The Discrete Fourier Transform (DFT)

Let's look at this using the trapezoidal rule. Recall that the trapezoidal rule says that an integral can be approximated by breaking the function into $N$ slices and summing the y-values like

\begin{align}
\int g(x) dx &\simeq h\left[\tfrac{1}{2}g(a) + \tfrac{1}{2}g(b) + \sum_{n=1}^{N-1}g(x_n)\right]
\end{align}

In our case, $h = \tfrac{L}{N}$ and $g(x_n) = f(x_n)\cdot\exp\left[-i\frac{2\pi kx_n}{L}\right]$. However, recall that we defined $f(x)$ to be periodic! Therefore $g(a) = g(b)$, and our discrete approximation for $y_k$ becomes

\begin{align}
\gamma_k = \frac{1}{N}\sum_{n=0}^{N-1}f(x_n)\exp\left(-i\frac{2\pi kx_n}{L}\right).
\end{align}

In the case that our function $f(x)$ is not continuous but is given to us already discretized with $y$values $y_n$ (which is very often the case!), we can rewrite this as

\begin{align}
c_k = \sum_{n=0}^{N-1}y_n\exp\left(-i\frac{2\pi kn}{N}\right),
\end{align}

where we are now labeling the Fourier coefficients as $c_k$ because this is a commonly used equation called the *discrete Fourier transform*. Notice that by convention, $c_k$ doesn't have the $\tfrac{1}{N}$ in front.

Just like the continuous counterpart, we can convert from $c_k$ back to $y_n$ using the discrete inverse Fourier transform, written as

\begin{align}
y_n = \frac{1}{N}\sum_{k=0}^{N-1}c_k\exp\left(i\frac{2\pi kn}{N}\right).
\end{align}

Note that our DFTs don't need to know the position of the samples along $x_n$! Besides this, there is one major difference staring us in the face when we compare the discrete Fourier transform and the Fourier Series: the sum in the former does not extend to infinity, while it does so in the latter.

One caveat is that our discrete Fourier transform, when using all the terms, is in fact exact! Even though we have a finite number of terms, we also have a finite number of sampling points. This means we are free to transform back and forth from the space domain ($y_n$) to the Fourier domain ($c_k$) and we won't lose any information. We call these kinds of transformations **lossless transformations**.

Another caveat is while we can reproduce our $y_n$ points exactly, we still have no-idea what is going on in between those points. For example, two functions that are different but happen to give the same $y_n$ will have the exact same DFT.


### Breakout 2: Write a DFT Function

Let's write a funtion that will perform the DFT and inverse DFT (IDFT) given an array of $y$ values and $c$ values. I've already written a DFT script for you, its your job to write the IDFT.

In [103]:
def dft(y):
    # get number of points
    N = len(y)
    
    # initialize empty array for coefficients
    c = np.empty(N, dtype=np.complex)
    
    # loop over coefficients, then loop over sum
    for k in range(N):
        for n in range(N):
            # Perform DFT sum
            c[k] += y[n] * np.exp(-1j*2*np.pi*k*n/N)
            
    return c

def idft(c):

            
    return y

1.
Use your functions to take the DFT of

* a sinc wave: $f(x) = \rm{sinc}(x)$
* a square wave: $f(x) = 1$
* a sawtooth wave: $f(x) = x$
* an exponential wave: $f(x) = 1/x$
* a sine wave: $f(x) = \sin(n\pi x)$

from $0 < x < L$, where you are free to choose $L$ as you want.

2.
Plot the Fourier coefficients. What shapes do you find in the Fourier domain as compared to the input domain?

Note that as we have written it, our DFT function needs to make $N^2$ computations every time we run it. Since computers can generally perform on the order of millions of computations per-second, we are limited to taking the DFT of signals with tens of thousands of elements if we want the program to finish in a reasonable amount of time.

### Interpretation

It is important to ask, what do the Fourier coefficients, $c_k$, in the DFT actually represent? If we inspect our DFT equation, what we see is that the coefficients are being multiplied by the complex exponentials, which are a compactified version of our sines and cosines. If we think back to our definition of the Fourier Series, we can see that the Fourier coefficients, $c_k$, are directly related to the $\alpha_k$ and $\beta_k$ coefficients of the sines and cosines. They represent, therefore, the amplitude of a sinusoid with a particular wavelength that is present in our data. The value of $k$ on the otherhand, represents the frequency of the sinusoid that is in the data.

### Breakout 3: Filtering and Detecting Periodicity

Let's use our DFT and IDFT functions to explore some data. You can find two data sets in the `data/` directory, one called `pitch.txt` and one called `sunspots.txt`. 

1.
Load in the data from `pitch.txt` and plot it. Then take its DFT and plot the square of the absolute magnitude of the Fourier coefficients $|c_k|^2$ (this is called the *power spectrum*), and see if you can interpret what it represents.

2.
What happens if you set some of the complex $c_k$ values to zero and then take its IDFT?

3.
Load in the data from `sunspots.txt` and plot it. Can you see a periodic signal? Take its DFT and see if you can identify the period of the strongest sinusoid in the data.

4.
Now repeat this using the `np.fft.fft` and `np.fft.ifft` functions and see what you get.